In [19]:
from src.finetuner import FineTuner
from src.inference import ModelInference
from src.preprocessor import Preprocessor
from transformers import (
    T5ForConditionalGeneration,
    AutoTokenizer,
    Seq2SeqTrainingArguments
)
from datasets import load_dataset

In [20]:
#constant
SAVE_PATH = f'../models/pt-indot5-TA_PT'
PRETRAINED_MODEL = "Wikidepia/IndoT5-base"

# Dataset

In [21]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

In [22]:
max_length = 128
text_col = 'input'
label_col = 'triplet'
preprocessor = Preprocessor('p1', tokenizer, max_length, text_col, label_col)

In [23]:
raw_dataset = load_dataset('csv', data_files={'train':'../Data/post-train/ASTE/train.csv'
                                                        , 'validation':'../Data/post-train/ASTE/dev.csv',
                                                        'test' : '../Data/post-train/ASTE/test.csv'})
preprocessor = Preprocessor('p1', tokenizer, max_length, text_col, label_col)
tokenized_dataset = raw_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=raw_dataset['train'].column_names)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-ecae22b8b998e95e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 749.96it/s]
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-ecae22b8b998e95e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-db44143562293862.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-ecae22b8b998e95e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ecf9d05cf85cad4f.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-ecae22b8b998e95e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-f4bfb634d23e25b1.arrow


In [24]:
tokenizer.decode(tokenized_dataset['train']['input_ids'][0])

'kamar saya ada kendala di ac tidak berfungsi optimal dan juga wifi koneksi kurang stabil</s>'

In [25]:
tokenizer.decode(tokenized_dataset['train']['labels'][0])

'(ac,tidak berfungsi optimal,NEG);(wifi koneksi,kurang stabil,NEG)</s>'

# Model Training

In [26]:
model = T5ForConditionalGeneration.from_pretrained(PRETRAINED_MODEL)

In [27]:
#training argument
training_args = Seq2SeqTrainingArguments(
    SAVE_PATH,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    resume_from_checkpoint=True,
    num_train_epochs=50,
    save_total_limit=2,
)

In [28]:
finetuner = FineTuner(model=model, save_path=SAVE_PATH, tokenizer=tokenizer, 
                      train_dataset=tokenized_dataset['train'], eval_dataset=tokenized_dataset['validation'])

cuda:0


In [29]:
finetuner.fine_tune(training_args)

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


cuda:0


  0%|          | 0/17750 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                     
  2%|▏         | 355/17750 [01:39<1:10:19,  4.12it/s]

{'eval_loss': 0.9452595710754395, 'eval_runtime': 6.2612, 'eval_samples_per_second': 154.443, 'eval_steps_per_second': 19.325, 'epoch': 1.0}


  3%|▎         | 501/17750 [02:38<1:01:48,  4.65it/s] 

...

{'loss': 0.0371, 'learning_rate': 2.8169014084507043e-07, 'epoch': 49.3}


                                                       
100%|██████████| 17750/17750 [1:35:44<00:00,  1.38s/it]

{'eval_loss': 0.2777468264102936, 'eval_runtime': 6.0533, 'eval_samples_per_second': 159.747, 'eval_steps_per_second': 19.989, 'epoch': 50.0}


100%|██████████| 17750/17750 [1:36:11<00:00,  3.08it/s]


{'train_runtime': 5771.7034, 'train_samples_per_second': 24.568, 'train_steps_per_second': 3.075, 'train_loss': 0.1517695594035404, 'epoch': 50.0}


# Inference

In [30]:
model = T5ForConditionalGeneration.from_pretrained(SAVE_PATH).to('cuda')

In [31]:
model_inference = ModelInference(batch_size=8, dataset=tokenized_dataset['test'], model=model, tokenizer=tokenizer)
pred_text = model_inference.inference()

100%|██████████| 117/117 [01:54<00:00,  1.03it/s]


In [32]:
test_dataset = raw_dataset['test']
test_dataset = test_dataset.add_column('triplet_model_prediction', pred_text)
test_dataset.to_csv('../Data/post-train/ASTE/model_inference.csv')

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 17.66ba/s]


265327

# Evaluate

In [33]:
import pandas as pd
from src.evaluator import Evaluator

In [34]:
df = pd.read_csv('../Data/post-train/ASTE/model_inference.csv')

In [35]:
evaluator = Evaluator(task_type='triplet')
raw_scores, all_labels, all_preds = evaluator.evaluate(df['triplet_model_prediction'].astype('str'), df['triplet'].astype('str'))

100%|██████████| 935/935 [00:00<00:00, 74731.30it/s]


In [36]:
raw_scores

{'precision': 0.746765974127793,
 'recall': 0.775020341741253,
 'f1': 0.7606308644440007}

# Inference with zili anotated data

In [1]:
from src.inference import ModelInference
from src.preprocessor import Preprocessor
from transformers import (
    T5ForConditionalGeneration,
    AutoTokenizer,
)
from datasets import load_dataset

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#constant
max_length = 20
text_col = 'content'
label_col = 'triplet_label'
preprocess_type = 'p00'
SAVE_PATH = f'../models/pt-indot5-TA_PT'
PRETRAINED_MODEL = "Wikidepia/IndoT5-base"
DATA_PATH = '../Data/quadruplet/quadruplet_2200-data_annottated_clean.csv'
inference_length = 128

In [3]:
model = T5ForConditionalGeneration.from_pretrained(SAVE_PATH)
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

In [6]:
preprocessor = Preprocessor(preprocess_type, tokenizer, max_length, text_col, label_col)
raw_dataset = load_dataset('csv', data_files=DATA_PATH)
tokenized_dataset = raw_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=raw_dataset['train'].column_names)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00,  7.51it/s]


Dataset csv downloaded and prepared to C:/Users/danendra/.cache/huggingface/datasets/csv/default-01756b12cba475bf/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 90.41it/s]


In [10]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2298
    })
})

In [11]:
tokenizer.decode(tokenized_dataset['train']['labels'][0])

'(hp, harga terlalu tinggi, negative)</s>'

In [12]:
tokenizer.decode(tokenized_dataset['train']['input_ids'][0])

'nder hpnya kamu pakai berapa lama? kayaknya jt masih terlalu tingi</s>'

In [13]:
model_inference = ModelInference(batch_size=8, dataset=tokenized_dataset['train'], model=model, 
                                 tokenizer=tokenizer, inference_len=inference_length)
pred_text = model_inference.inference()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 288/288 [02:19<00:00,  2.07it/s]


In [15]:
raw_dataset = raw_dataset['train'].add_column(f'{preprocess_type}_model_prediction', pred_text)
raw_dataset.to_csv('../Data/quadruplet/model_2200_data_semi_supervised_labels.csv')

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating CSV from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 47.03ba/s]


908982

# Evaluate

In [21]:
import pandas as pd
from src.evaluator import Evaluator
from src.postprocessor import PostProcessor
from src.utils import extract_triplet

In [25]:
df = pd.read_csv('../Data/quadruplet/model_2200_data_semi_supervised_labels.csv')
df['triplet_label'] = df['triplet_label'].apply(lambda x:x.lower())
df['p00_model_prediction'] = df['p00_model_prediction'].apply(lambda x:x.lower())
df.head(5)

,content,final_sentiment,baseline_aspect_category,tweet_type,clean_tweet,quadruplet,triplet_label,p00_model_prediction
0,@JUALAN_BASE @fleurlovincs Nder hpnya kamu pak...,neutral,delivery; produk;,SUBJECTIVE,Nder hpnya kamu pake brp lama? Kayaknya jt ms...,"(hp, harga terlalu tinggi, negative, produk)","(hp, harga terlalu tinggi, negative)","(hpnya,berapa lama,neg)"
1,@teh_manis__ @ezash coba cari di olx mba kalo ...,neutral,produk;,SUBJECTIVE,coba cari di olx mba kalo baru pertama kali m...,"(motor bekas, pertama kali, positive, produk)","(motor bekas, pertama kali, positive)","(motor,bekas,neg)"
2,@tanyakanrl Kalo mau beli second jangan di tok...,positive,delivery; produk;,SUBJECTIVE,Kalo mau beli second jangan di toko oren atau...,"(beli second, biar lebih aman, negative, produ...","(beli second, biar lebih aman, negative);(olx,...","(f,jangan di market place,neg)"
3,kalo jam 1 malem gabisa tidur pengen makan nas...,neutral,customerservice; produk;,SUBJECTIVE,kalo jam malem gabisa tidur pengen makan nasi ...,"(nasi goreng, enak, positive, produk)","(nasi goreng, enak, positive)","(nasi goreng,enak,pos)"
4,"@tirta_cipeng Di luar dunia fashion, saya seri...",neutral,produk;,SUBJECTIVE,"Di luar dunia fashion, saya sering banget nem...","(olx, reseller, negative, customerservice)","(olx, reseller, negative)","(budget,mengakui,pos)"


In [18]:
postprocessor = PostProcessor(use_postprocess=False)
evaluator = Evaluator(task_type='triplet', postprocessor=postprocessor)

In [19]:
raw_scores, all_labels, all_preds = evaluator.evaluate(pred_seqs=df['p00_model_prediction'],
                   gold_seqs=df['triplet_label'])

100%|██████████| 2298/2298 [00:00<00:00, 114761.93it/s]


post-process

In [31]:
sen_map = {
    'neg' : 'negative',
    'pos' : 'positive',
    'net' : 'neutral'
}
errors_idx = []
for i in range(len(df)):
    try:
        row = df.iloc[i]
        triplets = extract_triplet(row['p00_model_prediction'])
        new_triplet = []
        for triplet in triplets:
            aspect, opinion, sentiment = triplet
            sentiment = sen_map[sentiment]
            new_triplet.append(f"({aspect},{opinion},{sentiment})")
        new_triplet_str = ";".join(new_triplet)
        df.at[i, 'p00_model_prediction'] = new_triplet_str
    except:
        errors_idx.append(i)

In [34]:
clean_df = df.drop(index=errors_idx)

In [40]:
clean_df

,content,final_sentiment,baseline_aspect_category,tweet_type,clean_tweet,quadruplet,triplet_label,p00_model_prediction
17,Trus waktu kantorku hampir pindah di daerah ka...,neutral,produk;,SUBJECTIVE,Trus waktu kantorku hampir pindah di daerah ka...,"(mamikos, nyari kosan/kontrakan, olx, produk);...","(mamikos, nyari kosan/kontrakan, olx);(fesbuk,...",
23,@beautydyah Bisa kok hanya saja yang lebih dir...,positive,delivery; website&apps; payment; produk;,SUBJECTIVE,Bisa kok hanya saja yang lebih direkomendasik...,"(lebih direkomendasikan, keluar, centang biru,...","(lebih direkomendasikan, keluar, centang biru)","(biru,bisa,positive)"
24,"Btw ga cantumin foto soalnya bukan punyaku, jd...",negative,produk;,SUBJECTIVE,"Btw ga cantumin foto soalnya bukan punyaku, jd...","(penjual, ngajak cod-an, positive, produk)","(penjual, ngajak cod-an, positive)","(foto,ga cantumin,negative)"
25,"@babyjxxxoxo Wkwk lebay sih kl email, kecuali ...",neutral,produk;,SUBJECTIVE,"Wkwk lebay sih kl email, kecuali nomor wasaf ...","(pasang iklan, nomornya lupa hide, negative, w...","(pasang iklan, nomornya lupa hide, negative)","(email,berlebihan,negative)"
26,@SeputarTetangga Buat referensi coba liat2 aja...,neutral,price; produk;,SUBJECTIVE,Buat referensi coba liat2 aja harga rumah di ...,"(harga rumah, murah2, positive, produk); (kont...","(harga rumah, murah2, positive);(kontrakan, di...","(harga rumah,murah,positive)"
...,...,...,...,...,...,...,...,...
2293,@indomilkyourway @alfamart @alfamidi_ku @Indom...,positive,produk;,SUBJECTIVE,PINK BLOSSOM ENAK BGTIIII,"(pink blossom, enak bgt, positive, produk)","(pink blossom, enak bgt, positive)","(bgti,pink,positive);(bgti,enak bgti,positive)"
2294,shopee gw doang apa emang lagi error sih. tiap...,negative,produk;,SUBJECTIVE,shopee gw doang apa emang lagi error sih. tiap...,"(shopee, lagi error, negative, website&apps)","(shopee, lagi error, negative)","(shope,doang,negative)"
2295,@indomilkyourway @alfamart @alfamidi_ku @Indom...,positive,produk;,SUBJECTIVE,"Yang black latte enak banget sumpilll, plis j...","(black latte, enak banget sumpilll, positive, ...","(black latte, enak banget sumpilll, positive)","(varian,enak banget,positive);(varian,sumpil,p..."
2296,@sbtcon Di deskripsi link Shopee sllu tulis ka...,negative,produk;,SUBJECTIVE,Di deskripsi link Shopee sllu tulis kalo rusa...,"(packing, diluar tanggung jawab seller, negati...","(packing, diluar tanggung jawab seller, negative)","(seler,rusak,negative)"


In [41]:
raw_scores, all_labels, all_preds = evaluator.evaluate(pred_seqs=df['p00_model_prediction'],
                   gold_seqs=df['triplet_label'])

100%|██████████| 2298/2298 [00:00<00:00, 7800.50it/s]


In [43]:
clean_df.to_csv('../Data/quadruplet/model_2200_data_semi_supervised_labels.csv', index=False)